## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-15-07-08-04 +0000,bbc,"I'm 'disappointed but not done' with Putin, Tr...",https://www.bbc.com/news/articles/c1e02q12z32o
1,2025-07-15-07-01-41 +0000,bbc,Ex-Nigerian President Buhari to be buried at home,https://www.bbc.com/news/articles/c93kxl08glwo
2,2025-07-15-06-11-38 +0000,bbc,"Drivers offered up to £3,750 discount to buy e...",https://www.bbc.com/news/articles/cn5kpkypxp6o
3,2025-07-15-06-03-35 +0000,bbc,Nvidia says it will restart AI chip sales to C...,https://www.bbc.com/news/articles/cy8g22n32d0o
4,2025-07-15-05-30-36 +0000,bbc,Benn accuses Tories of making false promises t...,https://www.bbc.com/news/articles/ckg3pkn11lro


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2309/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
3,trump,15
54,ukraine,7
19,will,5
13,up,5
96,russia,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
10,2025-07-15-03-45-00 +0000,wsj,China’s economy showed resilience in a turbule...,https://www.wsj.com/world/china/chinas-economy...,47
46,2025-07-14-14-48-00 +0000,wsj,Trump is threatening to ramp up the U.S. trade...,https://www.wsj.com/economy/trade/european-uni...,47
15,2025-07-15-02-01-34 +0000,bbc,Trump threatens Russia with tariffs while unve...,https://www.bbc.com/news/articles/czdv20v9lp1o,40
14,2025-07-15-02-08-23 +0000,nyt,How Trump Changed His Tone on Putin and the Ru...,https://www.nytimes.com/2025/07/14/us/politics...,34
26,2025-07-14-20-35-11 +0000,nyt,Trump Threatens Russia With Sanctions and Vows...,https://www.nytimes.com/2025/07/14/us/politics...,33


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
10,47,2025-07-15-03-45-00 +0000,wsj,China’s economy showed resilience in a turbule...,https://www.wsj.com/world/china/chinas-economy...
46,32,2025-07-14-14-48-00 +0000,wsj,Trump is threatening to ramp up the U.S. trade...,https://www.wsj.com/economy/trade/european-uni...
15,25,2025-07-15-02-01-34 +0000,bbc,Trump threatens Russia with tariffs while unve...,https://www.bbc.com/news/articles/czdv20v9lp1o
58,20,2025-07-14-09-30-00 +0000,wsj,Food banks across the country were already str...,https://www.wsj.com/economy/food-banks-are-run...
32,12,2025-07-14-18-39-13 +0000,bbc,'Heart-breaking': Locals and visitors devastat...,https://www.bbc.com/news/articles/c4gd8d5jpy5o
27,10,2025-07-14-20-25-51 +0000,bbc,Motorists rescued as new flash flood warnings ...,https://www.bbc.com/news/videos/ce8zpggxp9eo
34,10,2025-07-14-17-11-33 +0000,bbc,MP charged for accusing Lesotho king of signin...,https://www.bbc.com/news/articles/cp86pznd65ko
9,9,2025-07-15-03-59-02 +0000,bbc,Meta to spend hundreds of billions to build AI...,https://www.bbc.com/news/articles/c1e02vx55wpo
16,9,2025-07-15-00-59-07 +0000,nyt,"Behind Trump’s Tough Talk on Russia and Putin,...",https://www.nytimes.com/2025/07/14/us/politics...
41,9,2025-07-14-15-48-36 +0000,nyt,BBC Ends Ties With ‘MasterChef’ Host Gregg Wal...,https://www.nytimes.com/2025/07/14/world/europ...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
